

# WARNING: The functionality herein should be considered experimental and unfinished.


In [ ]:
import safep
import alchemlyb
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from alchemlyb.parsing import namd
import pandas as pd

from IPython.display import display, Markdown

from alchemlyb.estimators import BAR
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# User parameters

In [ ]:
path='/path/to/fepouts'
filename='*.fepout'
temperature = 303.15

## IMPORTANT: Make sure the temperature above matches the temperature used to run the simulations.

In [ ]:
plt.rcParams['figure.dpi'] = 150
import os

RT = 0.00198720650096 * temperature

fepoutFiles = glob(path+filename)
totalSize = 0
for file in fepoutFiles:
    totalSize += os.path.getsize(file)
print(f"Will process {len(fepoutFiles)} fepout files.\nTotal size:{np.round(totalSize/10**9, 2)}GB")

In [ ]:
maxSize = 10**9 #Don't use the alchemlyb parser if larger than this size. (bytes)
decorrelate = True
detectEQ = True

#ML = fit PDF of discrepancies with a normal distribution maximum likelihood estimator. 
#LS = fit CDF of discrepancies with a normal distribution least-squares estimator
DiscrepancyFitting = 'LS' 

# For large data sets: read, decorrelate, save
This reduces RAM requirements between reading and decorrelating

Remember: pickles are not future-proof and should not be used for long-term data storage

In [ ]:
from alchemlyb.preprocessing import subsampling

In [ ]:
if totalSize>maxSize:
    method = 'dE'
    affix = f'decorrelated_{method}'

    pickles = []
    idx = 0

    for file in tqdm(fepoutFiles):
        df = readFiles([file])
        u_nk = u_nk_fromDF(df, temperature, 0, warnings=False)

        groups = u_nk.groupby('fep-lambda')
        decorr = pd.DataFrame([])
        for key, group in groups:
            test = subsampling.decorrelate_u_nk(group, method)
            decorr = decorr.append(test)
        u_nk = decorr
        pickle = f"{path}{affix}{idx:03d}.pkl"
        u_nk.to_pickle(pickle)
        pickles.append(pickle)
        idx +=1
        
    pickleDFs = []
    for pickle in pickles:
        pickleDFs.append(pd.read_pickle(pickle))

    u_nk = pd.concat(pickleDFs)

# Demonstration of equivalence between the above and below methods

In [ ]:
## Demonstrate that AFEP readFiles+u_nk_fromDF is identical to namd.extract_u_nk
## readFiles is more space efficient and can handle single files. Only reads each file once. Less input validation than alchemlyb.namd.
#u_nk_target = namd.extract_u_nk(fepoutFiles[0:5], temperature)
#df = readFiles(fepoutFiles[0:5])
#u_nk_test = u_nk_fromDF(df, temperature, 10000)
#np.all(u_nk_target.fillna(100)==u_nk_test.fillna(100))

# Small data sets can be read and decorrelated sequentially, if desired
See Shirts and Chodera (2008) for more details

"Statistically optimal analysis of samples from multiple equilibrium states" doi: 10.1063/1.2978177

In [ ]:
if totalSize < maxSize:
    from alchemlyb.preprocessing import subsampling

    u_nk = namd.extract_u_nk(fepoutFiles, temperature)
    
    affix=""
    
    if detectEQ:
        print("Detecting Equilibrium")
        affix = f"{affix}_AutoEquilibrium"
        groups = u_nk.groupby('fep-lambda')
        EQ = pd.DataFrame([])
        for key, group in groups:
            group = group[~group.index.duplicated(keep='first')]
            test = subsampling.equilibrium_detection(group, group.dropna(axis=1).iloc[:,-1])
            EQ = EQ.append(test)
        u_nk = EQ
    else:
        affix=f"{affix}_HardEquilibrium"
    
    if decorrelate:
        print("Decorrelating samples")
        method = 'dE'
        affix = f'{affix}_decorrelated_{method}'
        groups = u_nk.groupby('fep-lambda')
        decorr = pd.DataFrame([])
        for key, group in groups:
            test = subsampling.decorrelate_u_nk(group, method)
            decorr = decorr.append(test)
        u_nk = decorr
    else:
        affix = f'{affix}_unprocessed'

    
else:
    print(f"Warning: The files you are trying to read are quite large. Total size={totalSize}. Try reading and decorrelating (above) or change the maxSize parameter.")

# Carry out MBAR Fitting and Analyses

In [ ]:
u_nk = u_nk.sort_index(level=1)

In [ ]:
counts = pd.DataFrame()
for fl in set(u_nk.index.get_level_values(1)):
    counts[fl] = u_nk.loc[(slice(None), fl), :].count()

In [ ]:
countSeries = pd.Series(np.diagonal(counts.sort_index(axis=1)), index=counts.index)

In [ ]:
np.min(countSeries)

In [ ]:
plt.plot(countSeries)
plt.ylabel('Number of Samples')
plt.xlabel('Fep-lambda')
plt.title(f'{affix}_Number of samples per simulated lambda value')
plt.savefig(f'{path}{affix}_nSamples.png', dpi=600)

In [ ]:
groups = u_nk.groupby('fep-lambda')
lambdas=[key for key, group in groups]
print(lambdas)

## Demonstrate an autocorrelation plot

In [ ]:
from scipy.signal import correlate
ser = u_nk.loc[(slice(None), lambdas[0]), lambdas[1]]
flucs = ser - np.mean(ser)
corr = correlate(flucs, flucs, mode='same', method='direct')
corr = corr[len(corr)//2:]
plt.plot(corr)

# Extract key features from the MBAR fitting and get ΔG
Note: alchemlyb operates in units of kT by default. We multiply by RT to conver to units of kcal/mol.

In [ ]:
perWindow, cumulative = safep.do_estimation(u_nk) #Run the BAR estimator on the fep data
changeAndError = f'\u0394G = {np.round(forward[-1]*RT, 1)}\u00B1{np.round(errors[-1], 3)} kcal/mol'
print(changeAndError)

# Plot the change in free energy based on MBAR estimates

In [ ]:
# Cumulative change in kT
plt.errorbar(l, f, yerr=errors, marker='.')
plt.xlabel('lambda')
plt.ylabel('DeltaG(lambda) (kT)')
plt.title(f'Cumulative dG with accumulated errors {affix}\n{changeAndError}')
plt.savefig(f'{path}dG_cumulative_kT_{affix}.png', dpi=600)
plt.show()

# Cumulative change in kcal/mol
"""
plt.errorbar(l, f * RT, yerr=errors*RT, marker='.')
plt.xlabel('lambda')
plt.ylabel('DeltaG(lambda)(kcal/mol)')
plt.savefig(f'{path}dG_cumulative_kcal_per_mol_{affix}.png', dpi=600)
plt.show()
"""
# Per-window change in kT
plt.errorbar(l_mid, df, yerr=ddf, marker='.')
plt.xlabel('lambda')
plt.ylabel('Delta G per window (kT)')
plt.title(f'Per-Window dG with individual errors {affix}')
plt.savefig(f'{path}dG_{affix}.png', dpi=600)
plt.show()

# Per-window change in kT

plt.errorbar(l[1:-1], np.diff(df), marker='.')
plt.xlabel('lambda (L)')
plt.ylabel("ddG(L)")
plt.title(f'derivative of dG {affix}')
plt.savefig(f'{path}ddG_{affix}.png', dpi=600)
plt.show()


# Plot the estimated total change in free energy as a function of simulation time; contiguous subsets starting at t=0 ("Forward") and t=end ("Reverse")

In [ ]:
fig, convAx = plt.subplots(1,1)
forward, forward_error, backward, backward_error = safep.do_convergence(u_nk)
convAx = safep.convergence_plot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)
plt.savefig(f'{path}FEP_convergence.pdf')

In [ ]:
from tqdm import tqdm

In [ ]:
#Functions for bootstrapping estimates and generating confidence intervals
def bootStrapEstimate(u_nk, estimator='BAR', iterations=100, schedule=[10,20,30,40,50,60,70,80,90,100]):
    groups = u_nk.groupby('fep-lambda')

    if estimator == 'EXP':
        dGfs = {}
        dGbs = {}
        means = {}
    elif estimator == 'BAR':
        dGs = {}
        errs = {}
    else:
        raise ValueError(f"unknown estimator: {estimator}")

    for p in schedule:
        Fs = []
        Bs = []
        fs = []
        Gs = []
        #rs = []
        for i in np.arange(iterations):
            sampled = pd.DataFrame([])
            for key, group in groups:
                N = int(p*len(group)/100)
                if N < 1:
                    N=1
                rows = np.random.choice(len(group), size=N)
                test = group.iloc[rows,:]
                sampled = sampled.append(test)
            if estimator == 'EXP':
                l, l_mid, dG_f, dG_b = safep.get_EXP(pd.DataFrame(sampled))
                F = np.sum(dG_f)
                B = np.sum(-dG_b)
                Fs.append(F)
                Bs.append(B)
                Gs.append(np.mean([F,B]))
            elif estimator == 'BAR':
                tmpBar = BAR()
                tmpBar.fit(sampled)
                l, l_mid, f, df, ddf, errors = safep.get_BAR(tmpBar)
                fs.append(f.values[-1])
                #rs.append(errors[-1])

        if estimator == 'EXP':
            dGfs[p] = Fs
            dGbs[p] = Bs
            means[p] = Gs
        else:
            dGs[p] = fs
            #errs[p] = rs

    if estimator == 'EXP':
        fwd = pd.DataFrame(dGfs).melt().copy()
        bwd = pd.DataFrame(dGbs).melt().copy()
        alldGs = fwd.append(bwd)
        means = pd.DataFrame(means).melt().copy()
        return (alldGs, fwd, bwd, means)
    else:
        alldGs = pd.DataFrame(dGs).melt().copy()
        #allErrors = pd.DataFrame(errs).melt().copy()
        return alldGs

In [ ]:
estimator = 'BAR'
iterations = 100
schedule = tqdm([1, 10, 25, 50, 75, 100])
if estimator == 'BAR':
    alldGs = bootStrapEstimate(u_nk, estimator, iterations, schedule)
elif estimator == 'EXP':
    alldGs, fwd, bwd, means  = bootStrapEstimate(u_nk, estimator, iterations, schedule)
else:
    raise(f'Error: estimator {estimator} not known')


In [ ]:
if estimator=='EXP':
    fwd = fwd.set_index('variable')
    bwd = bwd.set_index('variable')
alldGs = alldGs.set_index('variable')

In [ ]:
interval=0.95
upper, lower, means = safep.get_empirical_CI(alldGs, interval)

upper, lower, means = getLimits(alldGs)

In [ ]:
#convergence_plot(u_nk, l)
#plt.plot(keys, upper, color='gray')
#plt.plot(keys, lower, color='gray')
keys = np.sort(list(set(alldGs.index)))
plt.fill_between(keys/100, upper, lower, color='gray', alpha=0.3, label=f"{interval*100}% confidence interval")
plt.plot(keys/100, means)
if estimator == 'EXP':
    plt.scatter(fwd.index/100, fwd.value, label='Forward Samples', s=2)
    plt.scatter(bwd.index/100, bwd.value, label='Backward Samples', s=2)
else:
    plt.scatter(alldGs.index/100, alldGs.value, label='BAR Estimates', s=2)
plt.legend()
plt.ylabel("Net dG (kT)")
plt.xlabel("Percent of data used")

plt.title(f'Bootstrapped Estimate: {np.round(means[-1]*RT,1)} +/- {np.round((upper[-1]-means[-1])*RT,1)}kcal/mol')

plt.savefig(f'{path}/convergence_{estimator}_{iterations}_{affix}.png', dpi=600)

In [ ]:
nbins = 10
if estimator == 'EXP':
    fvals, fbins, _ = plt.hist(fwd.loc[100], label='forward', bins=nbins, histtype='step')
    bvals, bbins, _ = plt.hist(bwd.loc[100], label='backward', bins=nbins, histtype='step')
    plt.plot(np.mean([fbins[1:], fbins[:-1]], axis=0), fvals)
    plt.plot(np.mean([bbins[1:], bbins[:-1]], axis=0), bvals)
    vals, bins, _ = plt.hist(alldGs.loc[100], label='Combined', bins=nbins, histtype='step')
    plt.plot(np.mean([bins[1:], bins[:-1]], axis=0), vals)
    plt.legend()
    plt.savefig(f'{path}/distribution_of_bootstrapped_samples_{estimator}_{iterations}_{affix}.png', dpi=600)
else:
    vals, bins, _ = plt.hist(alldGs.loc[100], label='BAR', bins=nbins)
    plt.plot(np.mean([bins[1:], bins[:-1]], axis=0), vals)
    plt.xlabel('dG in reduced units')
    plt.legend()
    plt.savefig(f'{path}/distribution_of_bootstrapped_samples_{estimator}_{iterations}_{affix}.png', dpi=600)

# Use an exponential estimator to assess residual discrepancies and check for hysteresis

In [ ]:
l, l_mid, dG_f, dG_b = safep.get_exponential(u_nk)

In [ ]:
hyst = dG_f + np.array(dG_b)
plt.vlines(l_mid, np.zeros(len(l_mid)), hyst, label="fwd - bwd", linewidth=2)

plt.legend()
plt.title(f'Fwd-bwd discrepancies by lambda {affix}')
plt.xlabel('Lambda')
plt.ylabel('Diff. in delta-G')
plt.savefig(f'{path}discrepancies_{affix}.png', dpi=600)

In [ ]:
plt.plot(l_mid, hyst)

In [ ]:
flucs = hyst - np.mean(hyst)
corr = correlate(flucs, flucs, mode='full')
corr = corr[len(corr)//2:]
plt.plot(corr)
#plt.savefig(f'

# OPTIONAL: Estimate and plot the Cumulative Density function (CDF) for the differences shown above.

In [ ]:
diff = dG_f + np.array(dG_b)
diff.sort()
X = diff
Y = np.arange(len(X))/len(X)

#fit a normal distribution to the existing data
#fitted = norm.fit(X)

In [ ]:
from scipy.special import erfc
from scipy.optimize import curve_fit as scipyFit
from scipy.stats import skew
from scipy.stats import norm

#Wrapper for fitting the normal CDF
def cumFn(x, m, s):
    r = norm.cdf(x, m, s)
    return r

def pdfFn(x,m,s):
    r = norm.pdf(x,m,s)
    return r

In [ ]:
#fit a normal distribution to the existing data

if DiscrepancyFitting == 'LS':
    fitted = scipyFit(cumFn, X, Y)[0] #Fit norm.cdf to (X,Y)
elif DiscrepancyFitting == 'ML':
    #fitted = scipyFit(pdfFn, X, Y)[0]
    fitted = norm.fit(X) # fit a normal distribution to X
else:
    raise("Error: Discrepancy fitting code not known. Acceptable values: ML or LS")
discrepancies = dG_f + np.array(dG_b)

dx = 0.01
cdfXnorm  = np.arange(np.min(X), np.max(X), dx)
cdfYnorm = norm.cdf(cdfXnorm, fitted[0], fitted[1])
cdfYexpected = norm.cdf(X, fitted[0], fitted[1])

#plot the data as they are (estimate the CDF) and the fitted cdf
fig, (cdfAx, cdfResid, pdfAx, pdfResid) = plt.subplots(4, 1, sharex=True)
plt.xlabel('Difference in delta-G')

cdfAx.scatter(X, Y, 2, label="Fwd bkd differences")
cdfAx.plot(cdfXnorm, cdfYnorm, label="Normal Distribution", color="orange")
cdfAx.set_ylabel("CDF")
cdfAx.legend()

#cdf Residuals
cdfResiduals = Y-cdfYexpected
cdfResid.plot(X, cdfResiduals)
cdfResid.set_ylabel("CDF residuals")

#pdf
dx = 0.01
dx = 0.01

binNum = 20
window = binNum
pdfY, pdfX = np.histogram(discrepancies, bins=binNum, density=True)
pdfX = (pdfX[1:]+pdfX[:-1])/2

pdfXnorm  = np.arange(np.min(X), np.max(X), dx)
pdfYnorm = norm.pdf(pdfXnorm, fitted[0], fitted[1])

pdfYexpected = norm.pdf(pdfX, fitted[0], fitted[1])

pdfAx.plot(pdfX, pdfY,  label="Estimated Distribution")
pdfAx.set_ylabel("PDF")
pdfAx.plot(pdfXnorm, pdfYnorm, label="Fitted Normal Distribution", color="orange")

#pdf residuals
pdfResiduals = pdfY-pdfYexpected
pdfResid.plot(pdfX, pdfResiduals)
pdfResid.set_ylabel("PDF residuals") 


fig.set_figheight(10)
if DiscrepancyFitting == 'LS':
    cdfAx.title.set_text(f"Least squares fitting of cdf(fwd-bkwd)\nSkewness: {np.round(skew(X),2)}\nFitted parameters: Mean={np.round(fitted[0],3)}, Stdv={np.round(fitted[1],3)}\nPopulation parameters: Mean={np.round(np.average(X),3)}, Stdv={np.round(np.std(X),3)}")
    plt.savefig(f"{path}LeastSquaresCDF{affix}.png", dpi=600)
elif DiscrepancyFitting == 'ML':
    cdfAx.title.set_text(f"Maximum likelihood fitting of fwd-bkwd\nFitted parameters: Mean={np.round(fitted[0],3)}, Stdv={np.round(fitted[1],3)}\nPopulation parameters: Mean={np.round(np.average(X),3)}, Stdv={np.round(np.std(X),3)}")
    plt.savefig(f"{path}MaximumLikelihood{affix}.png", dpi=600)

plt.show()

# OPTIONAL DIAGNOSTICS

# For looking at FEP data as a time series. Can be useful diagnostically.

In [ ]:
forwardMask = u_nk.copy()
offset = u_nk.columns[1] - u_nk.columns[0]

for col in forwardMask.columns:
    forwardMask[col].values[:] = 0

for x in set(u_nk.index.get_level_values(1)):
    forwardMask.loc[(slice(None),x),np.round(x+offset, 3)] = True

forwardMask = forwardMask.dropna(axis=1)

backwardMask = u_nk.copy()
offset = u_nk.columns[1] - u_nk.columns[0]

for col in backwardMask.columns:
    backwardMask[col].values[:] = 0

for x in set(u_nk.index.get_level_values(1)):
    backwardMask.loc[(slice(None),x),np.round(x+offset, 3)] = True

backwardMask = backwardMask.dropna(axis=1)

In [ ]:
u_nk.mask(forwardMask.astype(bool))

In [ ]:
l = 0.0000
l2 = 0.0125
forward = u_nk.loc[(slice(None), l), l2]
plt.plot(forward.index.get_level_values(0), forward, label="forward")
reverse = -1*u_nk.loc[(slice(None), l2), l]
plt.plot(reverse.index.get_level_values(0), reverse, label="reverse")
plt.legend()

In [ ]:
histF, edgesF = np.histogram(forward, density=True)
histR, edgesR = np.histogram(reverse, density=True)

plt.plot(np.mean([edgesF[1:], edgesF[:-1]], axis=0), histF, label ="forward")
plt.plot(np.mean([edgesR[1:], edgesR[:-1]], axis=0), histR, label="backward")
plt.legend()

In [ ]:

plt.plot(l_mid, dG_f)
plt.plot(l_mid, -dG_b)
plt.title(f"fwd and -bwd exponential estimations {affix}")
plt.xlabel("lambda")
plt.ylabel("dG")
plt.savefig(f"{path}exponentials.png", dpi=600)

In [ ]:
plt.plot(l_mid, np.cumsum(dG_f))
plt.plot(l_mid, -np.cumsum(dG_b))
plt.title(f"fwd and -bwd exponential estimations {affix}")
plt.xlabel("lambda")
plt.ylabel("dG")
plt.savefig(f"{path}exponentials_cumulative_{affix}.png", dpi=600)

# Using the exponential estimator to estimate an error

In [ ]:
import scipy.stats

In [ ]:
errors = []
for i in np.arange(len(dG_f)):
    errors.append(scipy.stats.sem([dG_f[i], -dG_b[i]]))

In [ ]:
err = np.linalg.norm(errors, ord=2)

print(f"The estimated cumulative error using the exponential estimator is: {np.round(err*RT,3)} kcal/mol")